<a href="https://colab.research.google.com/github/claudiaqw/deep-learning/blob/main/hungerGames_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab assignment: the hunger games

<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/breakfast.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/hamburger.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fruits.jpg" style="width:300px;height:300px;"></td>
</tr></table>

In this assignment we will face a challenging image classification problem, building a deep learning model that is able to classify different kinds of foods. Let the hunger games begin!

## Guidelines

Throughout this notebook you will find empty cells that you will need to fill with your own code. Follow the instructions in the notebook and pay special attention to the following symbols.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
You will need to solve a question by writing your own code or answer in the cell immediately below or in a different file, as instructed.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
This is a hint or useful observation that can help you solve this assignment. You should pay attention to these hints to better understand the assignment.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
This is an advanced exercise that can help you gain a deeper knowledge into the topic. Good luck!</font>

***

To avoid missing packages and compatibility issues you should run this notebook under one of the [recommended Deep Learning environment files](https://github.com/albarji/teaching-environments/tree/master/deeplearning), or make use of [Google Colaboratory](https://colab.research.google.com/). If you use Colaboratory make sure to [activate GPU support](https://colab.research.google.com/notebooks/gpu.ipynb).

The following code will embed any plots into the notebook instead of generating a new window:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Lastly, if you need any help on the usage of a Python function you can place the writing cursor over its name and press Caps+Shift to produce a pop-out with related documentation. This will only work inside code cells. 

Let's go!

## Data acquisition

We will use a food images dataset available at [Kaggle](https://www.kaggle.com/trolukovich/food11-image-dataset). To download it you will need to create a user account in Kaggle, and obtain your API credential by following the instructions on [this section](https://www.kaggle.com/trolukovich/food11-image-dataset). Once you have your credentials JSON file, you can inform this notebook of them by setting the appropriate enviroment variables, as follows

    import os

    os.environ["KAGGLE_USERNAME"] = "YOUR KAGGLE USERNAME HERE"
    os.environ["KAGGLE_KEY"] = "YOUR KAGGLE KEY HERE"
    
Once this is done, you will be able to download the dataset to this computer using the command

    !kaggle datasets download trolukovich/food11-image-dataset --unzip -p YOUR_LOCAL_FOLDER
    
where you should write a valid directory in your computer in "YOUR_LOCAL_FOLDER". If you are fine with downloading the data in the same folder as this notebook, just skip the `-p YOUR_LOCAL_FOLDER` part of the command.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Create a Kaggle account, obtain your credentials, and use the cell below to declare your Kaggle username and key variables, and to download the dataset to a local folder.
    
These credentials should remain secret to you. Remember to delete them before submitting this notebook!
</font>

***

In [1]:
####### INSERT YOUR CODE HERE

import os
credentials = {"username":"claudiaqw",
               "key":"7a4a42e84feb6a7da837ebf98550b12b"}

os.environ['KAGGLE_USERNAME']=credentials["username"]
os.environ['KAGGLE_KEY']=credentials["key"]

!kaggle datasets download trolukovich/food11-image-dataset --unzip

 99% 1.07G/1.08G [00:06<00:00, 170MB/s]
100% 1.08G/1.08G [00:06<00:00, 167MB/s]


Take now a look into the folder where you downloaded the data. You will find it is made up of three subfolders:

* **training**, containing the images to use to train the model.
* **validation**, containing additional images you could use as more training data, or for some kind of validation strategy such as Early Stopping.
* **evaluation**, containing the images you must use to test your model. Images in this folder can **only** be used to measure the model performance after the training procedure is completed.

Furthermore, within each one of these folders you will find one folder for each one of the 11 classes in this problem:

* Bread
* Dairy product
* Dessert
* Egg
* Fried food
* Meat
* Noodles-Pasta
* Rice
* Seafood
* Soup
* Vegetable-Fruit

This is a standard structure for organizing image datasets: one folder per class. To easen the following data processing steps, let us define some variables telling us where the data is located.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Create variables <b>TRAINDIR</b>, <b>VALDIR</b> and <b>TESTDIR</b> with the paths to the folders with the training, validation and evaluation data, respectively.
</font>

***

In [2]:
####### INSERT YOUR CODE HERE

TRAINDIR, VALDIR, TESTDIR = '/content/training', '/content/validation', '/content/evaluation'


Let's plot a random sample of training images from each class, using the ipyplot package. If you are running this notebook in Google Colab, you will need to install this package first with

    !pip install ipyplot

You can inspect each class by clicking the different tabs in the interface that will appear when running the following cell.

In [3]:
# !pip install ipyplot

In [4]:
# from glob import glob
# import ipyplot
# import numpy as np

# all_images = glob(f"{TRAINDIR}/*/*.jpg")  # Get all image paths
# np.random.shuffle(all_images)  # Randomize to show different images each run
# all_labels = [f.split("/")[-2] for f in all_images]  # Extract class names from path

# ipyplot.plot_class_tabs(all_images, all_labels, max_imgs_per_tab=6, img_width=300, force_b64=True)

### Class reduction

To make the problem more approachable for this exercise, we will focus on just six classes: `Bread`, `Dairy product`, `Dessert`, `Egg`, `Fried food` and `Meat`. To do so, we will delete from the downloaded data the folders from other classes.

In [5]:
from glob import glob
import os

valid_classes = {"Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat"}
datasets = {TRAINDIR, VALDIR, TESTDIR}

for dataset in datasets:
    for classdir in glob(f"{dataset}/*"):  # Find subfolders with classes
        if classdir.split("/")[-1] not in valid_classes:  # Ignore those in valid_classes
            print(f"Deleting {classdir}...")
            for fname in glob(f"{classdir}/*.jpg"):  # Remove each image file
                os.remove(fname)
            os.rmdir(classdir)  # Remove folder

Deleting /content/training/Rice...
Deleting /content/training/Soup...
Deleting /content/training/Noodles-Pasta...
Deleting /content/training/Seafood...
Deleting /content/training/Vegetable-Fruit...
Deleting /content/evaluation/Rice...
Deleting /content/evaluation/Soup...
Deleting /content/evaluation/Noodles-Pasta...
Deleting /content/evaluation/Seafood...
Deleting /content/evaluation/Vegetable-Fruit...
Deleting /content/validation/Rice...
Deleting /content/validation/Soup...
Deleting /content/validation/Noodles-Pasta...
Deleting /content/validation/Seafood...
Deleting /content/validation/Vegetable-Fruit...


## Image processing from files

This dataset of images is large, with images of larger resolution than in the tutorial MNIST problem, each one having different sizes and image ratios. Also, while for MNIST we had a keras function that prepared the dataset for us, this time we will need to do some loading and image processing work.

A convenient way to do this work is through the use of Keras `image_dataset_from_directory` function. This function creates a TensorFlow `Dataset` with the images in the directory, loading images dynamically only when the neural network needs to use them, and also allowing us to specify some useful preprocessing options.

For example, we can create such a `Dataset` with the data in the training folder:

In [6]:
from keras.preprocessing import image_dataset_from_directory

image_size = 32
batch_size = 64

train_dataset = image_dataset_from_directory(
    TRAINDIR, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
)

Found 6082 files belonging to 6 classes.


Note the parameters used to configure the dataset:

* The **directory** from which to load the images.
* An **image size** that will be used to resize all the images to a common resolution, here 32x32.
* The **size of the batches** of images to be generated. Note we define this parameter here instead in the network fit step, as the `Dataset` will make use of this information to keep in memory only a few batches of images at the same time in order to save memory.
* The **label mode**, that is, the encoding used for the labels. `categorical` means we will use one-hot encoding.

A `Dataset` object works like a Python generator, which means we can iterate over it to obtain batches of processed images. For instance, to get the first batch

In [7]:
for X_batch, y_batch in train_dataset:
    print(f"Shape of input batch: {X_batch.shape}")
    print(f"Shape of output batch: {y_batch.shape}")
    print(f"Input batch:\n{X_batch}")
    print(f"Output batch:\n{y_batch}")
    break

Shape of input batch: (64, 32, 32, 3)
Shape of output batch: (64, 6)
Input batch:
[[[[7.92500000e+01 5.32500000e+01 4.72500000e+01]
   [8.12500000e+01 4.92500000e+01 4.07500000e+01]
   [1.36250000e+02 1.39750000e+02 1.49750000e+02]
   ...
   [1.49000000e+02 1.57500000e+02 1.67500000e+02]
   [1.46250000e+02 1.51250000e+02 1.55750000e+02]
   [1.53000000e+02 1.56250000e+02 1.61750000e+02]]

  [[7.60000000e+01 5.95000000e+01 5.55000000e+01]
   [8.22500000e+01 4.70000000e+01 5.20000000e+01]
   [1.48000000e+02 1.54000000e+02 1.66750000e+02]
   ...
   [1.33250000e+02 1.38750000e+02 1.46250000e+02]
   [1.29250000e+02 1.36750000e+02 1.51250000e+02]
   [1.07250000e+02 1.11750000e+02 1.19250000e+02]]

  [[7.17500000e+01 6.27500000e+01 6.10000000e+01]
   [7.95000000e+01 4.30000000e+01 4.72500000e+01]
   [1.11750000e+02 1.14750000e+02 1.34000000e+02]
   ...
   [1.30000000e+02 1.38500000e+02 1.44750000e+02]
   [1.19000000e+02 1.21500000e+02 1.29750000e+02]
   [1.65500000e+02 1.67000000e+02 1.7075000

We can see that indeed the generator produces a tensor of the appropriate dimensions with the inputs for the neural network, and that the outputs have also been properly codified in one-hot form. However, there is still an issue with the data: the pixel values are in the range [0, 255], which might produce training problems. We will solve this later in the network definition. For now, let's move on and prepare a funcion that builds the training, validation and test datasets.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Create a function <b>create_datasets</b> that receives the following parameters:
    <ul>
      <li><b>traindir</b>: the directory where training images are located.</li>
      <li><b>valdir</b>: the directory where validation images are located.</li>
      <li><b>testdir</b>: the directory where test images are located.</li>
      <li><b>image_size</b>: the size that will be used to resize all the images to a common resolution.</li>
      <li><b>batch_size</b>: the size of the batches of images to be generated.</li>
    </ul>
    The function must create datasets for the training, validation and test directories, and return the three datasets created.

    return train_dataset, val_dataset, test_dataset
</font>

***

In [8]:
####### INSERT YOUR CODE HERE

def create_datasets(traindir=TRAINDIR, valdir=VALDIR, testdir=TESTDIR, image_size=32, batch_size=64):
  label_mode = 'categorical'
  
  train_dataset = image_dataset_from_directory(
    traindir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = label_mode
    )
  
  val_dataset = image_dataset_from_directory(
    valdir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = label_mode
    )
  
  test_dataset = image_dataset_from_directory(
    testdir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = label_mode
    )
  
  return train_dataset, val_dataset, test_dataset

Let's test if the function you just implemented works correctly

In [9]:
import tensorflow as tf

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=32, batch_size=512)
train_dataset_64, val_dataset_64, test_dataset_64 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=64, batch_size=256)
train_dataset_128, val_dataset_128, test_dataset_128 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=128, batch_size=128)
train_dataset_256, val_dataset_256, test_dataset_256 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=256, batch_size=64)


# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(val_dataset, tf.data.Dataset)
assert isinstance(test_dataset, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


Now that we have our datasets we can train a deep learning model using them! For illustration purposes, let's build an extremely simple convolutional network. Note how we have added a special pre-processing layer `Rescaling` that takes care of normalizing the data to the range [0, 1].

Be careful! This network will work, but has some flaws in its design you might want to fix in the network you will desing later in this notebook.

In [10]:
from numpy.random import seed
import random

seed(42)
tf.random.set_seed(42)

In [11]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.experimental.preprocessing import Rescaling

model = Sequential()
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model.add(Convolution2D(4, 3, activation='linear'))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

The `fit` method of a Keras model can receive a `Dataset` with training data, instead of a pair of tensors with (inputs, outputs). Since when building the `Dataset` we already specified the batch size, we don't need to do it now.

In [12]:
model.fit(train_dataset, epochs=1)

96/96 [==============================] - 44s 127ms/step - loss: 1.7360 - accuracy: 0.2501


Similarly, we can evaluate the performance of our model over our test `Dataset` as follows

In [13]:
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 122ms/step - loss: 2.2224 - accuracy: 0.1986
Loss 2.22, accuracy 19.9%


The accuracy might seem poor, but take into account we have used a very simple model and this problem has 6 classes. Will you be able to do better?

## Building your network

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Design a neural network that maximizes the accuracy over the test data. You can use the training and validation datasets for anything you like, but you can <b>only</b> use the test data to evaluate the model performance. You should obtain a network able to attain at least 40% accuracy over the test set.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Some tips and strategies that can help you optimize your network design:

    
- Make use of all the tricks you learned from previous notebooks: convolutional + pooling layers, ReLU activations, dropout... also make sure to use a good optimizer with an adequate loss function, as well as the correct activation for the output layer.
- Configuring the datasets to load the images with a larger size can significantly improve your performance. But be careful, you can also run out of memory (CUDA memory error) if they become too large! Note that for this problem a size larger than 256 might be too large.
- Start with networks with a small number of parameters, so you are able to check fast how well they work. Then you can make your network larger in three directions: larger input images, more layers and more kernels per convolutional layer or units per dense layer. If you use larger images make sure to add more Convolution+Pooling layers, so that only very small images (less than 10x10 pixels) arrive at the Flatten layer.
- If you see large differences in loss between your training data and your validation or test data, try increasing the Dropout probabilities, especially for the Dense layers.
- Make use of that validation data! For instance, use an <a href="https://keras.io/api/callbacks/early_stopping/">**EarlyStopping strategy**</a> to monitor the loss of these validation data, and stop when training when after a number of iterations such loss has not decreased. Configuring the EarlyStopping to restore the best weights found in the optimization is also useful.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
Other advanced strategies you can try are:

- Use **image augmentation techniques** to artifically create new training images. To do so, explore the rest of layers available in the <a href="https://keras.io/api/layers/preprocessing_layers/image_preprocessing/">Keras Image Preprocessing module</a>.
- Use <a href="https://keras.io/api/layers/normalization_layers/batch_normalization/">BatchNormalization</a> layers to improve the optimization procedure.
    
It you use all the tricks, it is possible to obtain more than 60% accuracy in the test set.

</font>

***

In [22]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.experimental.preprocessing import Rescaling

from keras.callbacks import EarlyStopping
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop,RandomTranslation, RandomRotation, RandomZoom
from keras.preprocessing.image import ImageDataGenerator

### Model 1: Changing optimizer

The first change in the original model is to replace the *sgd* optimizer by the *Adam*. As we can see, the model performance on test has improved.

In [ ]:
seed(42)
tf.random.set_seed(42)

In [ ]:
####### INSERT YOUR CODE HERE
pool_size = 2

model_1 = Sequential()
model_1.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_1.add(Convolution2D(4, 3, activation='linear'))
model_1.add(Flatten())
model_1.add(Dense(6, activation='sigmoid'))

model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
model_1.fit(train_dataset, epochs=1, validation_data = val_dataset)

96/96 [==============================] - 19s 182ms/step - loss: 1.8442 - accuracy: 0.2558 - val_loss: 1.6544 - val_accuracy: 0.2941


In [ ]:
loss, acc = model_1.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 5s 114ms/step - loss: 1.6408 - accuracy: 0.3159
Loss 1.64, accuracy 31.6%


### Model 2: adding a max pooling layer

A MaxPooling layer has been introduced after the convolution. The number of epochs during training has been increased.

---



In [ ]:
pool_size = 2

model_2 = Sequential()
model_2.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_2.add(Convolution2D(4, 3, activation='linear', strides=2))
model_2.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_2.add(Flatten())
model_2.add(Dense(6, activation='sigmoid'))

opt = Adam(learning_rate=0.01)
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_4 (Rescaling)      (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 4)         112       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 4)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 196)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 1182      
Total params: 1,294
Trainable params: 1,294
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_2.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
96/96 [==============================] - 20s 190ms/step - loss: 1.9338 - accuracy: 0.2055 - val_loss: 1.6844 - val_accuracy: 0.2737
Epoch 2/10
96/96 [==============================] - 19s 186ms/step - loss: 1.6721 - accuracy: 0.2879 - val_loss: 1.6418 - val_accuracy: 0.3041
Epoch 3/10
96/96 [==============================] - 19s 187ms/step - loss: 1.6236 - accuracy: 0.3209 - val_loss: 1.5910 - val_accuracy: 0.3325
Epoch 4/10
96/96 [==============================] - 19s 184ms/step - loss: 1.5809 - accuracy: 0.3398 - val_loss: 1.5648 - val_accuracy: 0.3558
Epoch 5/10
96/96 [==============================] - 19s 185ms/step - loss: 1.5553 - accuracy: 0.3557 - val_loss: 1.5271 - val_accuracy: 0.3676
Epoch 6/10
96/96 [==============================] - 19s 183ms/step - loss: 1.5156 - accuracy: 0.3777 - val_loss: 1.5026 - val_accuracy: 0.3795
Epoch 7/10
96/96 [==============================] - 19s 183ms/step - loss: 1.4949 - accuracy: 0.3796 - val_loss: 1.4910 - val_accuracy: 0.3752

In [ ]:
loss, acc = model_2.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 5s 115ms/step - loss: 1.4921 - accuracy: 0.3952
Loss 1.49, accuracy 39.5%


### Model 3: Changing activation functions

Th activation function in the last dense layer has been change from *sigmoid* to *softmax*

---

In [ ]:
pool_size = 2

model_3 = Sequential()
model_3.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_3.add(Convolution2D(8, 3, activation='relu', strides=1))
model_3.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_3.add(Convolution2D(16, 3, activation='relu', strides=1))
model_3.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_3.add(Flatten())
model_3.add(Dense(6, activation='softmax'))

model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_3.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_7 (Rescaling)      (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 30, 8)         224       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 8)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 13, 13, 16)        1168      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                

In [ ]:
model_3.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
96/96 [==============================] - 16s 156ms/step - loss: 1.6873 - accuracy: 0.2868 - val_loss: 1.5024 - val_accuracy: 0.3686
Epoch 2/10
96/96 [==============================] - 16s 160ms/step - loss: 1.5047 - accuracy: 0.3786 - val_loss: 1.4747 - val_accuracy: 0.3819
Epoch 3/10
96/96 [==============================] - 16s 158ms/step - loss: 1.4824 - accuracy: 0.3828 - val_loss: 1.4611 - val_accuracy: 0.3971
Epoch 4/10
96/96 [==============================] - 16s 159ms/step - loss: 1.4557 - accuracy: 0.4116 - val_loss: 1.4627 - val_accuracy: 0.4018
Epoch 5/10
96/96 [==============================] - 16s 158ms/step - loss: 1.4542 - accuracy: 0.4170 - val_loss: 1.4450 - val_accuracy: 0.4046
Epoch 6/10
96/96 [==============================] - 16s 158ms/step - loss: 1.4190 - accuracy: 0.4227 - val_loss: 1.4141 - val_accuracy: 0.4122
Epoch 7/10
96/96 [==============================] - 16s 159ms/step - loss: 1.4027 - accuracy: 0.4416 - val_loss: 1.4104 - val_accuracy: 0.4132

In [ ]:
loss, acc = model_3.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 5s 109ms/step - loss: 1.3956 - accuracy: 0.4338
Loss 1.4, accuracy 43.4%


### Model 4 
We have added another convolution but with relu activation, a rectified linear activation and dropout with probability 0.5





Now, we try to improve the network adding another convolution with 32 channels to generate and an relu activation, a dense layer with 50 units and rectified linear activation and a dropout with 50% of probability

In [ ]:
kernel_size = 3
model_4 = Sequential()
model_4.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_4.add(Convolution2D(8, (kernel_size, kernel_size), input_shape=(image_size, image_size, 3), activation='relu'))
model_4.add(Convolution2D(32, (kernel_size, kernel_size), activation="relu"))
model_4.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_4.add(Flatten())
model_4.add(Dense(50, activation="relu"))
model_4.add(Dropout(0.5))
model_4.add(Dense(6, activation='softmax'))

opt = Adam(learning_rate=0.01)
model_4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
model_4.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
96/96 [==============================] - 14s 136ms/step - loss: 1.8523 - accuracy: 0.2556
Epoch 2/10
96/96 [==============================] - 14s 134ms/step - loss: 1.5526 - accuracy: 0.3514
Epoch 3/10
96/96 [==============================] - 13s 127ms/step - loss: 1.5708 - accuracy: 0.3502
Epoch 4/10
96/96 [==============================] - 13s 128ms/step - loss: 1.5821 - accuracy: 0.3539
Epoch 5/10
96/96 [==============================] - 13s 125ms/step - loss: 1.5140 - accuracy: 0.3748
Epoch 6/10
96/96 [==============================] - 13s 127ms/step - loss: 1.4610 - accuracy: 0.4019
Epoch 7/10
96/96 [==============================] - 13s 127ms/step - loss: 1.4637 - accuracy: 0.4084
Epoch 8/10
96/96 [==============================] - 13s 127ms/step - loss: 1.4913 - accuracy: 0.3979
Epoch 9/10
96/96 [==============================] - 13s 128ms/step - loss: 1.4761 - accuracy: 0.3991
Epoch 10/10
96/96 [==============================] - 13s 127ms/step - loss: 1.4976 - accura

In [ ]:
loss, acc = model_4.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 4s 89ms/step - loss: 1.5514 - accuracy: 0.3478
Loss 1.55, accuracy 34.8%


As we can see, although the accuracy in train has decreased slightly, we have achieved a better one in test. 

### Model 5: increasing image size 64x64 images
We decided to try with larger input images

In [ ]:
image_size = 64
model_5 = Sequential()
model_5.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_5.add(Convolution2D(8, 3, input_shape=(image_size, image_size, 3), activation='linear'))
model_5.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_5.add(Convolution2D(32, 3, activation="relu"))
model_5.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_5.add(Convolution2D(64, 3, activation="relu"))
model_5.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_5.add(Convolution2D(128, 3, activation="relu"))
model_5.add(MaxPooling2D(pool_size=pool_size, strides=1))
model_5.add(Dropout(0.5))
model_5.add(Flatten())
model_5.add(Dense(200, activation="relu"))
model_5.add(Dropout(0.5))
model_5.add(Dense(6, activation='softmax'))

model_5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_5.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_12 (Rescaling)     (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 62, 62, 8)         224       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 31, 31, 8)         0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 29, 29, 32)        2336      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 6, 6, 64)        

In [ ]:
model_5.fit(train_dataset_64, epochs=10, validation_data=val_dataset_64)

Epoch 1/10
24/24 [==============================] - 16s 496ms/step - loss: 1.7302 - accuracy: 0.2469 - val_loss: 1.6033 - val_accuracy: 0.2922
Epoch 2/10
24/24 [==============================] - 15s 488ms/step - loss: 1.5980 - accuracy: 0.3171 - val_loss: 1.4501 - val_accuracy: 0.3918
Epoch 3/10
24/24 [==============================] - 15s 494ms/step - loss: 1.4800 - accuracy: 0.3854 - val_loss: 1.4189 - val_accuracy: 0.4241
Epoch 4/10
24/24 [==============================] - 15s 493ms/step - loss: 1.4342 - accuracy: 0.4121 - val_loss: 1.3753 - val_accuracy: 0.4417
Epoch 5/10
24/24 [==============================] - 15s 497ms/step - loss: 1.4099 - accuracy: 0.4278 - val_loss: 1.3583 - val_accuracy: 0.4602
Epoch 6/10
24/24 [==============================] - 15s 500ms/step - loss: 1.3813 - accuracy: 0.4354 - val_loss: 1.3408 - val_accuracy: 0.4758
Epoch 7/10
24/24 [==============================] - 15s 502ms/step - loss: 1.3404 - accuracy: 0.4690 - val_loss: 1.3311 - val_accuracy: 0.4611

In [ ]:
loss, acc = model_5.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

9/9 [==============================] - 4s 12ms/step - loss: 1.2325 - accuracy: 0.5309
Loss 1.23, accuracy 53.1%


### Model 6:

This we are going to ...

This model reaches a high performance over training set, but not on validation wich means is overfitted

In [ ]:
model_6 = Sequential()
model_6.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_6.add(Convolution2D(8, 2, input_shape=(64, 64), activation='relu'))
model_6.add(MaxPooling2D(pool_size=pool_size, strides=1, padding='valid'))
model_6.add(Convolution2D(32, 2, activation='relu'))
model_6.add(MaxPooling2D(pool_size=2, strides=2, padding='valid'))
model_6.add(Convolution2D(64, 2, activation='relu'))
model_6.add(MaxPooling2D(pool_size=2, strides=2, padding='valid'))
model_6.add(Convolution2D(128, 2, activation='relu'))
model_6.add(MaxPooling2D(pool_size=2, strides=2, padding='valid'))
model_6.add(Convolution2D(256, 2, activation='relu'))
model_6.add(MaxPooling2D(pool_size=2, strides=1, padding='valid'))
model_6.add(Dropout(0.5))
model_6.add(Flatten())
model_6.add(Dense(1024, activation='relu'))
model_6.add(Dropout(0.5))
model_6.add(Dense(200, activation='relu'))
model_6.add(Dense(6, activation='softmax'))


opt = Adam(learning_rate=0.01)
model_6.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_6.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_13 (Rescaling)     (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 63, 63, 8)         104       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 62, 62, 8)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 61, 61, 32)        1056      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 29, 29, 64)        8256      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 14, 14, 64)      

In [ ]:
model_6.fit(train_dataset_64, epochs=20, validation_data=val_dataset_64)

Epoch 1/20
24/24 [==============================] - 18s 560ms/step - loss: 1.7249 - accuracy: 0.2330 - val_loss: 1.5863 - val_accuracy: 0.3188
Epoch 2/20
24/24 [==============================] - 16s 521ms/step - loss: 1.5658 - accuracy: 0.3486 - val_loss: 1.4941 - val_accuracy: 0.3676
Epoch 3/20
24/24 [==============================] - 16s 516ms/step - loss: 1.5019 - accuracy: 0.3581 - val_loss: 1.5286 - val_accuracy: 0.3757
Epoch 4/20
24/24 [==============================] - 16s 517ms/step - loss: 1.4821 - accuracy: 0.3841 - val_loss: 1.4172 - val_accuracy: 0.4222
Epoch 5/20
24/24 [==============================] - 16s 519ms/step - loss: 1.4132 - accuracy: 0.4198 - val_loss: 1.3857 - val_accuracy: 0.4307
Epoch 6/20
24/24 [==============================] - 16s 519ms/step - loss: 1.3639 - accuracy: 0.4445 - val_loss: 1.3551 - val_accuracy: 0.4635
Epoch 7/20
24/24 [==============================] - 16s 520ms/step - loss: 1.3340 - accuracy: 0.4572 - val_loss: 1.3285 - val_accuracy: 0.4587

In [ ]:
loss, acc = model_6.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

9/9 [==============================] - 5s 29ms/step - loss: 1.2359 - accuracy: 0.5357
Loss 1.24, accuracy 53.6%


### Model 7 (este es el que pensamos con 64)

In [ ]:
image_size = 64
model_7 = Sequential()
model_7.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_7.add(Convolution2D(4, 3, input_shape=(image_size, image_size, 3), strides=2, activation= 'relu'))
model_7.add(Convolution2D(16, 3,strides=1, activation= 'relu'))
model_7.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model_7.add(Convolution2D(32, 3,strides=1, activation= 'relu'))
model_7.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model_7.add(Flatten())
model_7.add(Dense(6, activation='softmax'))

model_7.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_7.summary()

In [ ]:
model_7.fit(train_dataset_64, epochs=20, validation_data=val_dataset_64)

Epoch 1/20
96/96 [==============================] - 11s 106ms/step - loss: 1.7289 - accuracy: 0.2546
Epoch 2/20
96/96 [==============================] - 11s 106ms/step - loss: 1.5786 - accuracy: 0.3330
Epoch 3/20
96/96 [==============================] - 11s 107ms/step - loss: 1.5501 - accuracy: 0.3519
Epoch 4/20
96/96 [==============================] - 11s 107ms/step - loss: 1.4830 - accuracy: 0.3987
Epoch 5/20
96/96 [==============================] - 11s 107ms/step - loss: 1.4454 - accuracy: 0.4164
Epoch 6/20
96/96 [==============================] - 11s 106ms/step - loss: 1.4167 - accuracy: 0.4370
Epoch 7/20
96/96 [==============================] - 11s 104ms/step - loss: 1.4064 - accuracy: 0.4383
Epoch 8/20
96/96 [==============================] - 11s 106ms/step - loss: 1.3930 - accuracy: 0.4433
Epoch 9/20
96/96 [==============================] - 11s 106ms/step - loss: 1.3772 - accuracy: 0.4496
Epoch 10/20
96/96 [==============================] - 11s 105ms/step - loss: 1.3482 - accura

In [ ]:
loss, acc = model_8.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 4s 95ms/step - loss: 1.3564 - accuracy: 0.4700
Loss 1.36, accuracy 47.0%


In [ ]:
model_8.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_11 (Rescaling)     (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 31, 31, 4)         112       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 29, 29, 8)         296       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 12, 12, 16)        1168      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 576)             

### Model 8: 50.5 accuracy with 50 epochs + overfitted

Another dense layer close to the output layer

In [ ]:
image_size = 64
model_8 = Sequential()
model_8.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_8.add(Convolution2D(4, 3, input_shape=(image_size, image_size, 3), strides=2, activation= 'relu'))
model_8.add(Convolution2D(8, 3,strides=1, activation= 'relu'))
model_8.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model_8.add(Convolution2D(16, 3,strides=1, activation= 'relu'))
model_8.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model_8.add(Flatten())
model_8.add(Dense(128, activation='relu'))
model_8.add(Dropout(0.5))
model_8.add(Dense(6, activation='softmax'))

model_8.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
model_8.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 31, 4)         112       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 8)         296       
Total params: 408
Trainable params: 408
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_8.fit(train_dataset_64, epochs=50)

Epoch 1/50
96/96 [==============================] - 45s 129ms/step - loss: 1.7353 - accuracy: 0.2451
Epoch 2/50
96/96 [==============================] - 13s 130ms/step - loss: 1.5906 - accuracy: 0.3284
Epoch 3/50
96/96 [==============================] - 13s 129ms/step - loss: 1.5534 - accuracy: 0.3384
Epoch 4/50
96/96 [==============================] - 14s 131ms/step - loss: 1.5153 - accuracy: 0.3703
Epoch 5/50
96/96 [==============================] - 14s 131ms/step - loss: 1.4707 - accuracy: 0.4075
Epoch 6/50
96/96 [==============================] - 14s 132ms/step - loss: 1.4404 - accuracy: 0.4051
Epoch 7/50
96/96 [==============================] - 14s 131ms/step - loss: 1.4326 - accuracy: 0.4054
Epoch 8/50
96/96 [==============================] - 13s 130ms/step - loss: 1.4009 - accuracy: 0.4342
Epoch 9/50
96/96 [==============================] - 14s 130ms/step - loss: 1.3500 - accuracy: 0.4468
Epoch 10/50
96/96 [==============================] - 14s 131ms/step - loss: 1.3645 - accura

In [ ]:
loss, acc = model_8.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 4s 89ms/step - loss: 1.6145 - accuracy: 0.4952
Loss 1.61, accuracy 49.5%


### Model 9: AlexNet

In [ ]:
image_size = 256
model_9 = Sequential()
model_9.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_9.add(Convolution2D(96, 7, input_shape=(image_size, image_size, 3), strides=1, activation= 'relu'))
model_9.add(Convolution2D(128, 5, strides=2, activation= 'relu'))
model_9.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model_9.add(Convolution2D(256, 5, strides=1, activation= 'relu'))
model_9.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model_9.add(Convolution2D(256, 3, strides=1, activation= 'relu'))
model_9.add(Convolution2D(512, 3, strides=1, activation= 'relu'))
model_9.add(Convolution2D(512, 3, strides=1, activation= 'relu'))
model_9.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model_9.add(Flatten())
model_9.add(Dense(512, activation='relu'))
model_9.add(Dense(6, activation='softmax'))

model_9.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
model_9.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_22 (Rescaling)     (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 250, 250, 96)      14208     
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 123, 123, 128)     307328    
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 61, 61, 128)       0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 57, 57, 256)       819456    
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 28, 28, 256)       0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 26, 26, 256)     

In [ ]:
model_9.fit(train_dataset_256, epochs=10)

In [ ]:
loss, acc = model_9.evaluate(test_dataset_256)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

### Model 10: overfitted

The size is 128

In [ ]:
image_size = 128
model_10 = Sequential()
model_10.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_10.add(Convolution2D(32, 3, strides=1, activation= 'relu'))
model_10.add(MaxPooling2D(pool_size=2, strides=2))
model_10.add(Convolution2D(64, 5,strides=1, activation= 'relu'))
model_10.add(MaxPooling2D(pool_size=2, strides=2))
model_10.add(Convolution2D(128, 3,strides=1, activation= 'relu'))
model_10.add(MaxPooling2D(pool_size=2, strides=2))
model_10.add(Convolution2D(256, 3,strides=1, activation= 'relu'))
model_10.add(MaxPooling2D(pool_size=2, strides=2))
model_10.add(Flatten())
model_10.add(Dense(4096, activation='relu'))
model_10.add(Dropout(0.5))
model_10.add(Dense(256, activation='relu'))
model_10.add(Dropout(0.5))
model_10.add(Dense(6, activation='softmax'))

model_10.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_10.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_34 (Rescaling)     (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_136 (Conv2D)          (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_130 (MaxPoolin (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_137 (Conv2D)          (None, 59, 59, 64)        51264     
_________________________________________________________________
max_pooling2d_131 (MaxPoolin (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_138 (Conv2D)          (None, 27, 27, 128)       73856     
_________________________________________________________________
max_pooling2d_132 (MaxPoolin (None, 13, 13, 128)     

In [ ]:
model_10.fit(train_dataset_128, epochs=20, validation_data=val_dataset_128)

Epoch 1/20
96/96 [==============================] - 19s 178ms/step - loss: 1.7070 - accuracy: 0.2477
Epoch 2/20
96/96 [==============================] - 18s 180ms/step - loss: 1.5485 - accuracy: 0.3381
Epoch 3/20
96/96 [==============================] - 18s 183ms/step - loss: 1.4580 - accuracy: 0.3848
Epoch 4/20
96/96 [==============================] - 18s 181ms/step - loss: 1.4394 - accuracy: 0.3955
Epoch 5/20
96/96 [==============================] - 18s 181ms/step - loss: 1.3852 - accuracy: 0.4278
Epoch 6/20
96/96 [==============================] - 18s 182ms/step - loss: 1.3381 - accuracy: 0.4646
Epoch 7/20
96/96 [==============================] - 19s 184ms/step - loss: 1.2766 - accuracy: 0.5011
Epoch 8/20
96/96 [==============================] - 19s 189ms/step - loss: 1.2134 - accuracy: 0.5241
Epoch 9/20
96/96 [==============================] - 19s 188ms/step - loss: 1.2138 - accuracy: 0.5221
Epoch 10/20
96/96 [==============================] - 19s 187ms/step - loss: 1.2140 - accura

In [ ]:
loss, acc = model_10.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 141ms/step - loss: 2.5931 - accuracy: 0.5256
Loss 2.59, accuracy 52.6%


### Model 11

In [ ]:
image_size = 128
model_11 = Sequential()
model_11.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_11.add(Convolution2D(32, 3, strides=1, activation= 'relu'))
model_11.add(MaxPooling2D(pool_size=2, strides=2))
model_11.add(Convolution2D(64, 5,strides=1, activation= 'relu'))
model_11.add(MaxPooling2D(pool_size=2, strides=2))
model_11.add(Convolution2D(128, 3,strides=1, activation= 'relu'))
model_11.add(MaxPooling2D(pool_size=2, strides=2))
model_11.add(Convolution2D(256, 3,strides=1, activation= 'relu'))
model_11.add(MaxPooling2D(pool_size=2, strides=2))
model_11.add(Flatten())
model_11.add(Dense(4096, activation='relu'))
model_11.add(Dropout(0.5))
model_11.add(Dense(1024, activation='relu'))
model_11.add(Dropout(0.5))
model_11.add(Dense(256, activation='relu'))
model_11.add(Dropout(0.5))
model_11.add(Dense(6, activation='softmax'))

model_11.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_11.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_36 (Rescaling)     (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_144 (Conv2D)          (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_138 (MaxPoolin (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_145 (Conv2D)          (None, 59, 59, 64)        51264     
_________________________________________________________________
max_pooling2d_139 (MaxPoolin (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_146 (Conv2D)          (None, 27, 27, 128)       73856     
_________________________________________________________________
max_pooling2d_140 (MaxPoolin (None, 13, 13, 128)     

In [ ]:
model_11.fit(train_dataset_128, epochs=20, validation_data=val_dataset_128)

Epoch 1/20
96/96 [==============================] - 19s 181ms/step - loss: 1.7504 - accuracy: 0.2252
Epoch 2/20
96/96 [==============================] - 19s 186ms/step - loss: 1.6319 - accuracy: 0.2878
Epoch 3/20
96/96 [==============================] - 19s 187ms/step - loss: 1.5670 - accuracy: 0.3242
Epoch 4/20
96/96 [==============================] - 19s 187ms/step - loss: 1.5108 - accuracy: 0.3452
Epoch 5/20
96/96 [==============================] - 19s 186ms/step - loss: 1.4491 - accuracy: 0.3735
Epoch 6/20
96/96 [==============================] - 19s 187ms/step - loss: 1.4016 - accuracy: 0.4208
Epoch 7/20
96/96 [==============================] - 19s 189ms/step - loss: 1.4188 - accuracy: 0.4257
Epoch 8/20
96/96 [==============================] - 18s 183ms/step - loss: 1.3143 - accuracy: 0.4645
Epoch 9/20
96/96 [==============================] - 18s 184ms/step - loss: 1.2339 - accuracy: 0.5032
Epoch 10/20
96/96 [==============================] - 20s 195ms/step - loss: 1.3008 - accura

In [ ]:
loss, acc = model_11.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 146ms/step - loss: 2.4466 - accuracy: 0.5005
Loss 2.45, accuracy 50.0%


### Model 12

In [ ]:
image_size = 128
model_12 = Sequential()
model_12.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_12.add(Convolution2D(32, 3, strides=1, activation= 'relu'))
model_12.add(MaxPooling2D(pool_size=2, strides=2))
model_12.add(Convolution2D(64, 3,strides=1, activation= 'relu'))
model_12.add(MaxPooling2D(pool_size=2, strides=2))
model_12.add(Convolution2D(128, 3,strides=1, activation= 'relu'))
model_12.add(MaxPooling2D(pool_size=2, strides=2))
model_12.add(Convolution2D(256, 3,strides=1, activation= 'relu'))
model_12.add(MaxPooling2D(pool_size=2, strides=2))
model_12.add(Flatten())
model_12.add(Dense(2048, activation='relu')) #con 4096 llegabamos a 0.67 en train y 0.45 en test
model_12.add(Dropout(0.6))
model_12.add(Dense(1024, activation='relu'))
model_12.add(Dropout(0.6))
model_12.add(Dense(256, activation='relu'))
model_12.add(Dropout(0.5))
model_12.add(Dense(6, activation='softmax'))

model_12.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_12.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_38 (Rescaling)     (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_152 (Conv2D)          (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_146 (MaxPoolin (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_153 (Conv2D)          (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_147 (MaxPoolin (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_154 (Conv2D)          (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_148 (MaxPoolin (None, 14, 14, 128)     

In [ ]:
model_12.fit(train_dataset_128, epochs=15)

Epoch 1/15
96/96 [==============================] - 21s 198ms/step - loss: 1.7536 - accuracy: 0.2326
Epoch 2/15
96/96 [==============================] - 18s 180ms/step - loss: 1.6229 - accuracy: 0.2908
Epoch 3/15
96/96 [==============================] - 18s 180ms/step - loss: 1.5985 - accuracy: 0.3016
Epoch 4/15
96/96 [==============================] - 18s 180ms/step - loss: 1.5370 - accuracy: 0.3371
Epoch 5/15
96/96 [==============================] - 18s 174ms/step - loss: 1.5004 - accuracy: 0.3498
Epoch 6/15
96/96 [==============================] - 18s 176ms/step - loss: 1.4635 - accuracy: 0.3803
Epoch 7/15
96/96 [==============================] - 18s 181ms/step - loss: 1.3991 - accuracy: 0.4164
Epoch 8/15
96/96 [==============================] - 19s 185ms/step - loss: 1.4147 - accuracy: 0.4118
Epoch 9/15
96/96 [==============================] - 18s 176ms/step - loss: 1.3839 - accuracy: 0.4296
Epoch 10/15
96/96 [==============================] - 18s 175ms/step - loss: 1.3656 - accura

In [ ]:
loss, acc = model_12.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 5s 133ms/step - loss: 1.5043 - accuracy: 0.5043
Loss 1.5, accuracy 50.4%


### Model 13: ImageGenerator and Early Stopping

Same model but data augmentation techniques 

In [ ]:
image_size = 128
model_13 = Sequential()
model_13.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_13.add(RandomFlip(mode="horizontal"))
model_13.add(RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)))
model_13.add(RandomRotation(factor=(-0.1, 0.1)))
model_13.add(RandomTranslation(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)))
model_13.add(Convolution2D(32, 3, strides=1, activation= 'relu'))
model_13.add(MaxPooling2D(pool_size=2, strides=2))
model_13.add(Convolution2D(64, 3,strides=1, activation= 'relu'))
model_13.add(MaxPooling2D(pool_size=2, strides=2))
model_13.add(Convolution2D(128, 3,strides=1, activation= 'relu'))
model_13.add(MaxPooling2D(pool_size=2, strides=2))
model_13.add(Convolution2D(256, 3,strides=1, activation= 'relu'))
model_13.add(MaxPooling2D(pool_size=2, strides=2))
model_13.add(Flatten())
model_13.add(Dense(2048, activation='relu'))
model_13.add(Dropout(0.6))
#model_21.add(Dense(1024, activation='relu'))
#model_21.add(Dropout(0.6))
model_13.add(Dense(256, activation='relu'))
model_13.add(Dropout(0.5))
model_13.add(Dense(6, activation='softmax'))


model_13.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_13.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_14 (Rescaling)     (None, 128, 128, 3)       0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 128, 128, 3)       0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 128, 128, 3)       0         
_________________________________________________________________
random_translation (RandomTr (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 63, 63, 32)      

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=20)

history = model_13.fit(train_dataset_128, epochs=50, validation_data=val_dataset_128, callbacks=[callback])

Epoch 1/50
48/48 [==============================] - 23s 402ms/step - loss: 1.7587 - accuracy: 0.2224 - val_loss: 1.6300 - val_accuracy: 0.3245
Epoch 2/50
48/48 [==============================] - 19s 357ms/step - loss: 1.6297 - accuracy: 0.2922 - val_loss: 1.6450 - val_accuracy: 0.3283
Epoch 3/50
48/48 [==============================] - 19s 361ms/step - loss: 1.5410 - accuracy: 0.3572 - val_loss: 1.4336 - val_accuracy: 0.4156
Epoch 4/50
48/48 [==============================] - 20s 373ms/step - loss: 1.4985 - accuracy: 0.3847 - val_loss: 1.4465 - val_accuracy: 0.4236
Epoch 5/50
48/48 [==============================] - 20s 370ms/step - loss: 1.4544 - accuracy: 0.4059 - val_loss: 1.4506 - val_accuracy: 0.4250
Epoch 6/50
48/48 [==============================] - 20s 390ms/step - loss: 1.4209 - accuracy: 0.4185 - val_loss: 1.4011 - val_accuracy: 0.4739
Epoch 7/50
48/48 [==============================] - 21s 407ms/step - loss: 1.3920 - accuracy: 0.4356 - val_loss: 1.4659 - val_accuracy: 0.4663

In [ ]:
loss, acc = model_13.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

17/17 [==============================] - 5s 200ms/step - loss: 0.9942 - accuracy: 0.6483
Loss 0.994, accuracy 64.8%


### Model 14: Huge last model

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

model_14 = Sequential()
model_14.add(Rescaling(scale=1./255, input_shape=(128, 128, 3)))
model_14.add(RandomFlip(mode="horizontal"))
model_14.add(RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)))
model_14.add(RandomRotation(factor=(-0.1, 0.1)))
model_14.add(RandomTranslation(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)))

model_14.add(Conv2D(32, (3, 3), padding="same", input_shape=(128, 128, 3)))
model_14.add(Activation("relu"))
model_14.add(BatchNormalization())
model_14.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))
# (CONV => RELU) * 2 => POOL
model_14.add(Conv2D(64, (3, 3), padding="same"))
model_14.add(Activation("relu"))
#model.add(BatchNormalization(axis=chanDim))
model_14.add(BatchNormalization())
model_14.add(Conv2D(64, (3, 3), padding="same"))
model_14.add(Activation("relu"))
model_14.add(BatchNormalization())
model_14.add(MaxPooling2D(pool_size=(2, 2)))
model_14.add(Dropout(0.5))
# (CONV => RELU) * 2 => POOL
model_14.add(Conv2D(128, (3, 3), padding="same"))
model_14.add(Activation("relu"))
model_14.add(BatchNormalization())
model_14.add(Conv2D(128, (3, 3), padding="same"))
model_14.add(Activation("relu"))
model_14.add(BatchNormalization())
model_14.add(MaxPooling2D(pool_size=(2, 2)))
model_14.add(Dropout(0.5))

model_14.add(Conv2D(256, (3, 3), padding="same"))
model_14.add(Activation("relu"))
model_14.add(BatchNormalization())
model_14.add(MaxPooling2D(pool_size=(2, 2)))
model_14.add(Dropout(0.5))

# first (and only) set of FC => RELU layers
model_14.add(Flatten())
model_14.add(Dense(5000))
model_14.add(Activation("relu"))
model_14.add(BatchNormalization())
model_14.add(Dense(500))
model_14.add(Activation("relu"))
model_14.add(BatchNormalization())
model_14.add(Dropout(0.5))
# use a *softmax* activation for single-label classification
# and *sigmoid* activation for multi-label classification
model_14.add(Dense(6))
model_14.add(Activation("softmax"))

model_14.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_14.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
activation_9 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
activation_10 (Activation)   (None, 64, 64, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 64, 64, 64)      

In [ ]:
model_14.fit(train_dataset_128, epochs=50, validation_data=val_dataset_128)

Epoch 1/50
48/48 [==============================] - 32s 542ms/step - loss: 2.5393 - accuracy: 0.2667 - val_loss: 3.2329 - val_accuracy: 0.2244
Epoch 2/50
48/48 [==============================] - 27s 535ms/step - loss: 1.7110 - accuracy: 0.3920 - val_loss: 4.2824 - val_accuracy: 0.1309
Epoch 3/50
48/48 [==============================] - 28s 550ms/step - loss: 1.5540 - accuracy: 0.4424 - val_loss: 4.2788 - val_accuracy: 0.1347
Epoch 4/50
48/48 [==============================] - 28s 548ms/step - loss: 1.4434 - accuracy: 0.4921 - val_loss: 1.6822 - val_accuracy: 0.4279
Epoch 5/50
48/48 [==============================] - 28s 546ms/step - loss: 1.3053 - accuracy: 0.5218 - val_loss: 1.7886 - val_accuracy: 0.4227
Epoch 6/50
48/48 [==============================] - 28s 544ms/step - loss: 1.2111 - accuracy: 0.5495 - val_loss: 1.6337 - val_accuracy: 0.4322
Epoch 7/50
48/48 [==============================] - 28s 547ms/step - loss: 1.1558 - accuracy: 0.5806 - val_loss: 1.5215 - val_accuracy: 0.4720

KeyboardInterrupt: ignored

In [ ]:
loss, acc = model_14.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

## Transfer learning

While designing your own network might produce some nice results, it is generally better to transfer the knowledge available in a pre-trained network. This not only can produce better results, but also saves a lot of design time. The [Keras Applications](https://keras.io/api/applications/) module contains several network designs ready to use. For instance, to exploit the famous VGG16 network we do

In [15]:
from keras.applications import VGG16

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

58892288/58889256 [==============================] - 0s 0us/step


By default all Keras Applications networks are preloaded with the weights that were obtained from training the network over the [ImageNet dataset](http://www.image-net.org/). To adapt the network to our problem we need to specify the shape of our input images, and also remove the output layers (top) of the original network, since we have a different number of classes.

Now, how do we do transfer learning over this network? We will show here how to implement the bottleneck features strategy. First, we will mark the VGG16 model as **non-trainable**, so that the weights remain frozen

In [16]:
vgg16_model.trainable = False

Now we will build a neural network that includes the VGG16 model as one of its "layers". Since the VGG16 was trained with an specific way of normalizing the images, we will need to normalize our images in the same way. Conveniently, Keras also provides a function for doing VGG16-style normalization.

In [17]:
from keras.applications.vgg16 import preprocess_input

We can try this with some image ir our dataset

In [18]:
for X_batch, _ in train_dataset:
    break
    
print(f"Before normalizing: {X_batch[0, :3, :3, :]}")
print(f"After normalizing: {preprocess_input(X_batch)[0, :3, :3, :]}")

Before normalizing: [[[ 62.5   38.75  25.25]
  [ 97.5   88.    87.5 ]
  [119.   109.5  109.5 ]]

 [[ 48.25  26.    21.  ]
  [ 39.25  18.25  15.25]
  [ 45.75  25.75  16.25]]

 [[116.    97.5   79.  ]
  [115.5  100.25  78.25]
  [ 60.    55.25  53.5 ]]]
After normalizing: [[[-78.689     -78.029     -61.18     ]
  [-16.439003  -28.779     -26.18     ]
  [  5.560997   -7.2789993  -4.6800003]]

 [[-82.939     -90.779     -75.43     ]
  [-88.689     -98.529     -84.43     ]
  [-87.689     -91.029     -77.93     ]]

 [[-24.939003  -19.279      -7.6800003]
  [-25.689003  -16.529      -8.18     ]
  [-50.439003  -61.529     -63.68     ]]]


The normalization required by VGG16 involves swapping the order of color channels (RGB -> BGR) and substracting the mean values over the ImageNet dataset for each color channel separately. Fortunately the `preprocess_input` function does all the work for us. Furthermore, we can plug this function as the first layer of our network, similarly to the `Rescaling` we used before. We can do this with a `Lambda` layer, which allows building a layer out of any (differentiable!) function. Let's start our model with this layer.

In [19]:
from keras.layers import Lambda

model = Sequential()
model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

After this, we can add the whole VGG16 network as a layer, and our custom trainable layers after it. Note this is an overly simple model with some mistakes introduced; a real transfer learning network would have a better design.

In [20]:
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 32, 32, 3)         0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


Notice how in the model summary we can see the whole network has millions of parameters, but since we have frozen the VGG16 part, only a few thousand parameters will be trained: those in the Dense layer.

We can now compile and train this model in the usual way

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])
model.fit(train_dataset, epochs=1)

loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 136ms/step - loss: 157.6190 - accuracy: 0.2367
Loss 1.58e+02, accuracy 23.7%


<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Using the bottleneck strategy, implement a network doing transfer learning from VGG16. If you do it properly, you should be able to obtain better results than with your previously designed network, at least a 80% of accuracy over the test set.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Some tips and strategies that can help you optimize your network design:

    
- Include one or more Dense layers with the appropriate activation functions before the output layer.
- Try using a [GlobalAveragePooling layer](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) instead of a Flatten layer. This layer computes an average of all pixel values for each channel, and sometimes performs better than a regular Flatten.
- And remember all the tricks from the previous exercise!
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
You can further improve the network accuracy with the following ideas

- Use the PRO strategies from the previous exercise.
- Try other pre-trained networks from <a href="https://keras.io/api/applications/">Keras Applications</a>, such as ResNet, Xception or EfficientNet.
- Use more advanced transfer learning strategies, like fine-tuning or a combination of bottleneck features and fine-tuning.
   
If you use all the tricks, it is possible to obtain more than 90% accuracy in the test set.

</font>

***

### Model: vgg16

In [25]:
model_vgg16 = Sequential()
model_vgg16.add(Lambda(preprocess_input, input_shape=(32, 32, 3)))
model_vgg16.add(vgg16_model)
model_vgg16.add(Flatten())
model_vgg16.add(Dense(128, activation='relu'))
model_vgg16.add(Dense(6, activation='softmax'))

model_vgg16.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_vgg16.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 32, 32, 3)         0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 774       
Total params: 14,781,126
Trainable params: 66,438
Non-trainable params: 14,714,688
_________________________________________________________________


In [26]:
model_vgg16.fit(train_dataset, epochs=10, validation_data= val_dataset)

Epoch 1/10
96/96 [==============================] - 18s 173ms/step - loss: 12.4963 - accuracy: 0.3155 - val_loss: 4.8476 - val_accuracy: 0.4018
Epoch 2/10
96/96 [==============================] - 17s 172ms/step - loss: 3.6560 - accuracy: 0.4664 - val_loss: 3.8530 - val_accuracy: 0.4189
Epoch 3/10
96/96 [==============================] - 18s 176ms/step - loss: 2.1970 - accuracy: 0.5391 - val_loss: 3.4546 - val_accuracy: 0.4160
Epoch 4/10
96/96 [==============================] - 17s 172ms/step - loss: 1.4519 - accuracy: 0.6252 - val_loss: 3.2545 - val_accuracy: 0.4293
Epoch 5/10
96/96 [==============================] - 18s 175ms/step - loss: 1.0506 - accuracy: 0.6995 - val_loss: 3.1346 - val_accuracy: 0.4279
Epoch 6/10
96/96 [==============================] - 17s 171ms/step - loss: 0.7899 - accuracy: 0.7542 - val_loss: 3.1389 - val_accuracy: 0.4274
Epoch 7/10
96/96 [==============================] - 18s 178ms/step - loss: 0.6152 - accuracy: 0.7964 - val_loss: 3.0815 - val_accuracy: 0.443

In [27]:
loss, acc = model_vgg16.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 132ms/step - loss: 3.1444 - accuracy: 0.4657
Loss 3.14, accuracy 46.6%


En esta nueva red reemplazamos la cpa Flatten por GlobalAveragePooling2D

In [28]:
vgg16_model = VGG16(include_top=False, input_shape=(128, 128, 3))
vgg16_model.trainable = False

model_vgg16 = Sequential()
model_vgg16.add(Lambda(preprocess_input, input_shape=(128, 128, 3)))
model_vgg16.add(vgg16_model)
model_vgg16.add(GlobalAveragePooling2D())
model_vgg16.add(Dense(128, activation='relu'))
model_vgg16.add(Dense(6, activation='softmax'))

model_vgg16.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_vgg16.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 774       
Total params: 14,781,126
Trainable params: 66,438
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model_vgg16.fit(train_dataset_128, epochs=10, validation_data= val_dataset_128)

Epoch 1/10
48/48 [==============================] - 43s 714ms/step - loss: 5.7730 - accuracy: 0.3728 - val_loss: 1.6179 - val_accuracy: 0.6063
Epoch 2/10
48/48 [==============================] - 25s 480ms/step - loss: 1.2322 - accuracy: 0.6510 - val_loss: 1.2329 - val_accuracy: 0.6613
Epoch 3/10
48/48 [==============================] - 25s 485ms/step - loss: 0.8343 - accuracy: 0.7275 - val_loss: 1.1457 - val_accuracy: 0.6779
Epoch 4/10
48/48 [==============================] - 26s 502ms/step - loss: 0.6075 - accuracy: 0.7935 - val_loss: 1.1088 - val_accuracy: 0.6940
Epoch 5/10
48/48 [==============================] - 25s 495ms/step - loss: 0.5099 - accuracy: 0.8290 - val_loss: 1.0490 - val_accuracy: 0.7040
Epoch 6/10
48/48 [==============================] - 26s 510ms/step - loss: 0.3887 - accuracy: 0.8726 - val_loss: 1.0478 - val_accuracy: 0.7026
Epoch 7/10
48/48 [==============================] - 26s 511ms/step - loss: 0.3156 - accuracy: 0.9039 - val_loss: 1.0491 - val_accuracy: 0.7135

In [ ]:
loss, acc = model_vgg16.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

17/17 [==============================] - 9s 417ms/step - loss: 1.1150 - accuracy: 0.7130
Loss 1.12, accuracy 71.3%


hemos mejorado el accuracy en el testset pero es evidente el overfitting dado que ealcanzamos un accuracy en training del 90% que se reduce en un 20 en el test set. Por esta razon agregaremos capas Dropout

In [32]:
vgg16_model = VGG16(include_top=False, input_shape=(128, 128, 3))
vgg16_model.trainable = False

model_vgg16 = Sequential()
model_vgg16.add(Lambda(preprocess_input, input_shape=(128, 128, 3)))
model_vgg16.add(vgg16_model)
model_vgg16.add(GlobalAveragePooling2D())
model_vgg16.add(Dense(256, activation='relu'))
model_vgg16.add(Dropout(0.6))
model_vgg16.add(Dense(128, activation='relu'))
model_vgg16.add(Dropout(0.6))
model_vgg16.add(Dense(6, activation='softmax'))

model_vgg16.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_vgg16.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)              

In [ ]:
model_vgg16.fit(train_dataset_128, epochs=40, validation_data= val_dataset_128)

Epoch 1/40
48/48 [==============================] - 30s 587ms/step - loss: 8.7273 - accuracy: 0.2573 - val_loss: 1.3162 - val_accuracy: 0.5085
Epoch 2/40
48/48 [==============================] - 28s 550ms/step - loss: 1.9594 - accuracy: 0.3453 - val_loss: 1.3658 - val_accuracy: 0.5052
Epoch 3/40
48/48 [==============================] - 26s 513ms/step - loss: 1.6269 - accuracy: 0.3856 - val_loss: 1.2916 - val_accuracy: 0.5527
Epoch 4/40
48/48 [==============================] - 27s 530ms/step - loss: 1.5059 - accuracy: 0.4249 - val_loss: 1.2242 - val_accuracy: 0.5754
Epoch 5/40
48/48 [==============================] - 27s 535ms/step - loss: 1.4208 - accuracy: 0.4719 - val_loss: 1.1478 - val_accuracy: 0.6167
Epoch 6/40
48/48 [==============================] - 27s 518ms/step - loss: 1.3548 - accuracy: 0.5058 - val_loss: 1.0871 - val_accuracy: 0.6162
Epoch 7/40
48/48 [==============================] - 28s 544ms/step - loss: 1.2716 - accuracy: 0.5373 - val_loss: 1.0375 - val_accuracy: 0.6456

In [ ]:
loss, acc = model_vgg16.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

17/17 [==============================] - 7s 298ms/step - loss: 0.7450 - accuracy: 0.7594
Loss 0.745, accuracy 75.9%


Se ha mitigado el overfitting, sin embargo, el accuracy ha suido pero aun hay sobreentrenamineto por lo cual acudimos a técnicas de Data Augmentation

In [46]:
vgg16_model = VGG16(include_top=False, input_shape=(128, 128, 3))
vgg16_model.trainable = False

model_vgg16 = Sequential()
model_vgg16.add(Rescaling(scale=1./255, input_shape=(128, 128, 3)))
model_vgg16.add(RandomFlip(mode="horizontal"))
model_vgg16.add(RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)))
# model_vgg16.add(RandomRotation(factor=(-0.1, 0.1)))
model_vgg16.add(RandomTranslation(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)))
model_vgg16.add(Lambda(preprocess_input, input_shape=(128, 128, 3)))

model_vgg16.add(vgg16_model)

model_vgg16.add(GlobalAveragePooling2D())
model_vgg16.add(Dense(256, activation='relu'))
model_vgg16.add(Dropout(0.6))
model_vgg16.add(Dense(128, activation='relu'))
model_vgg16.add(Dropout(0.5))
model_vgg16.add(Dense(6, activation='softmax'))

model_vgg16.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_vgg16.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_9 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
rescaling_9 (Rescaling)      (None, 128, 128, 3)       0         
_________________________________________________________________
random_flip_16 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_flip_17 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_zoom_8 (RandomZoom)   (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_8 (RandomRot (None, 128, 128, 3)       0         
_________________________________________________________________
random_translation_8 (Random (None, 128, 128, 3)     

In [47]:
model_vgg16.fit(train_dataset_128, epochs=50, validation_data= val_dataset_128)

Epoch 1/50
48/48 [==============================] - 27s 499ms/step - loss: 1.8590 - accuracy: 0.2359 - val_loss: 1.6033 - val_accuracy: 0.3776
Epoch 2/50
48/48 [==============================] - 26s 507ms/step - loss: 1.6256 - accuracy: 0.3251 - val_loss: 1.4423 - val_accuracy: 0.4445
Epoch 3/50
48/48 [==============================] - 26s 506ms/step - loss: 1.5354 - accuracy: 0.3818 - val_loss: 1.3289 - val_accuracy: 0.4981
Epoch 4/50
48/48 [==============================] - 26s 510ms/step - loss: 1.4652 - accuracy: 0.4127 - val_loss: 1.2555 - val_accuracy: 0.5266
Epoch 5/50
48/48 [==============================] - 26s 508ms/step - loss: 1.4124 - accuracy: 0.4484 - val_loss: 1.1828 - val_accuracy: 0.5602
Epoch 6/50
48/48 [==============================] - 29s 566ms/step - loss: 1.3537 - accuracy: 0.4744 - val_loss: 1.1462 - val_accuracy: 0.5750
Epoch 7/50
48/48 [==============================] - 28s 551ms/step - loss: 1.3439 - accuracy: 0.4817 - val_loss: 1.1504 - val_accuracy: 0.5787

In [48]:
loss, acc = model_vgg16.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

17/17 [==============================] - 9s 448ms/step - loss: 0.9818 - accuracy: 0.6469
Loss 0.982, accuracy 64.7%


### Model ResNet

In [ ]:
from keras.applications import ResNet

resnet152 = ResNet152(include_top=False, input_shape=(64, 64, 3))
resnet152.trainable = False

In [ ]:
model_resnet = Sequential()
model_resnet.add(Lambda(preprocess_input, input_shape=(64, 64, 3)))
model_resnet.add(resnet152)
model_resnet.add(Flatten())
model_resnet.add(Dense(4096, activation='relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(512, activation='relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(128, activation='relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(6, activation='softmax'))

model_resnet.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
model_resnet.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_5 (Lambda)            (None, 64, 64, 3)         0         
_________________________________________________________________
resnet152v2 (Functional)     (None, 2, 2, 2048)        58331648  
_________________________________________________________________
flatten_7 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 4096)              33558528  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               2097664   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)              

In [ ]:
model_resnet.fit(train_dataset_64, epochs=20, validation_data=val_dataset_64)

Epoch 1/20
24/24 [==============================] - 32s 819ms/step - loss: 124892.1936 - accuracy: 0.1977 - val_loss: 1.7527 - val_accuracy: 0.2372
Epoch 2/20
24/24 [==============================] - 20s 698ms/step - loss: 21.8636 - accuracy: 0.2538 - val_loss: 1.7402 - val_accuracy: 0.2372
Epoch 3/20
24/24 [==============================] - 21s 741ms/step - loss: 4.0723 - accuracy: 0.2455 - val_loss: 1.7370 - val_accuracy: 0.2372
Epoch 4/20
24/24 [==============================] - 21s 746ms/step - loss: 2.2222 - accuracy: 0.2511 - val_loss: 1.7358 - val_accuracy: 0.2372
Epoch 5/20
24/24 [==============================] - 21s 742ms/step - loss: 13.8523 - accuracy: 0.2473 - val_loss: 1.7360 - val_accuracy: 0.2372
Epoch 6/20
24/24 [==============================] - 21s 744ms/step - loss: 3.8347 - accuracy: 0.2459 - val_loss: 1.7359 - val_accuracy: 0.2372
Epoch 7/20
24/24 [==============================] - 21s 747ms/step - loss: 1.8053 - accuracy: 0.2540 - val_loss: 1.7359 - val_accuracy:

In [ ]:
loss, acc = model_resnet.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

### Model: ResNet 2

In [ ]:
from keras.layers import GlobalAveragePooling2D

model_resnet = Sequential()


# model_resnet.add(Lambda(preprocess_input, input_shape=(64, 64, 3)))
model_resnet.add(resnet152)
model_resnet.add(GlobalAveragePooling2D())
model_resnet.add(Dense(128, activation='relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(6, activation='softmax'))


In [59]:
from keras.applications import InceptionResNetV2

inception = InceptionResNetV2(include_top=False, input_shape=(256, 256, 3))
inception.trainable = False

In [63]:
model_inception = Sequential()
model_inception.add(Rescaling(scale=1./255, input_shape=(256, 256, 3)))
model_inception.add(RandomFlip(mode="horizontal"))
# model_inception.add(RandomFlip(mode="vertical"))
model_inception.add(RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)))
model_inception.add(RandomRotation(factor=(-0.1, 0.1)))
model_inception.add(RandomTranslation(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)))

model_inception.add(inception)
model_inception.add(GlobalAveragePooling2D())
model_inception.add(Dense(512, activation='relu'))
model_inception.add(Dropout(0.6))
model_inception.add(Dense(128, activation='relu'))
model_inception.add(Dropout(0.5))
model_inception.add(Dense(6, activation='softmax'))

model_inception.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_inception.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_14 (Rescaling)     (None, 256, 256, 3)       0         
_________________________________________________________________
random_flip_23 (RandomFlip)  (None, 256, 256, 3)       0         
_________________________________________________________________
random_zoom_13 (RandomZoom)  (None, 256, 256, 3)       0         
_________________________________________________________________
random_rotation_13 (RandomRo (None, 256, 256, 3)       0         
_________________________________________________________________
random_translation_13 (Rando (None, 256, 256, 3)       0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_15  (None, 1536)            

In [64]:
earlystopping = EarlyStopping(
    monitor = 'val_loss', 
    verbose = 1, 
    patience = 30,
    restore_best_weights=True
)

In [ ]:
model_inception.fit(train_dataset_256, epochs=50, validation_data=val_dataset_256, callbacks=[earlystopping])

Epoch 1/50
96/96 [==============================] - 87s 797ms/step - loss: 1.7899 - accuracy: 0.3532 - val_loss: 0.8982 - val_accuracy: 0.6926
Epoch 2/50
96/96 [==============================] - 75s 767ms/step - loss: 1.1620 - accuracy: 0.5747 - val_loss: 0.7754 - val_accuracy: 0.7372
Epoch 3/50
96/96 [==============================] - 75s 766ms/step - loss: 1.0792 - accuracy: 0.6176 - val_loss: 0.7029 - val_accuracy: 0.7652
Epoch 4/50
96/96 [==============================] - 75s 765ms/step - loss: 1.0126 - accuracy: 0.6445 - val_loss: 0.6666 - val_accuracy: 0.7742
Epoch 5/50
96/96 [==============================] - 75s 765ms/step - loss: 0.9681 - accuracy: 0.6677 - val_loss: 0.6415 - val_accuracy: 0.7680
Epoch 6/50
96/96 [==============================] - 75s 767ms/step - loss: 0.9368 - accuracy: 0.6767 - val_loss: 0.6296 - val_accuracy: 0.7813
Epoch 7/50
96/96 [==============================] - 75s 767ms/step - loss: 0.9356 - accuracy: 0.6768 - val_loss: 0.6268 - val_accuracy: 0.7842

In [ ]:
loss, acc = model_inception.evaluate(test_dataset_256)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

## Summary of results

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Write in the following cell a short report explaining what network designs you tried, and what test accuracies you obtained. What worked and what didn't? What have you learned from this exercise?
</font>

***

Example results table

|Model|Image processing|Neural network model|Training strategy|Test accuracy|
|--------------------|----------------|--------------------|-----------------|-------------|
|1|Size 32x32, batch size 256|Conv(4, linear) + Flatten + Dense(6, sigmoid)|Train from scratch|xx%|
|2|Size 32x32, batch size 256|Conv(4, linear) + MaxPooling(2) + Flatten + Dense(6, sigmoid)|Train from scratch|xx%|
|3|Size 32x32, batch size 256|Conv(8, relu) + MaxPooling(2) + Conv(16, relu) + MaxPooling(2) + Flatten + Dense(6, softmax)|Train from scratch|xx%|
|4|Size 32x32, batch size 256|Conv(8, relu) + Conv(32, relu) + MaxPooling(2) + Flatten + Dense(50, relu) + Dropout(0,5) + Dense(6, sigmoid)|Train from scratch|xx%|
|5|Size 64x64, batch size 128|Conv(8, linear) + MaxPooling(2) + Conv(32, relu) + MaxPooling(2) + Conv(64, relu) + MaxPooling(2) + Flatten + Dense(200, relu) + Dropout(0,5) + Dense(6, softmax)|Train from scratch|xx%|
|6|Size 64x64, batch size 128|Conv(8, relu) + MaxPooling(2) + Conv(32, relu) + MaxPooling(2) + Conv(64, relu) + MaxPooling(2) + Conv(128, relu) + MaxPooling(2) + Conv(256, relu) + MaxPooling(2) + Dropout(0,5) + Flatten + Dense(1024, relu) + Dropout(0,5) + Dense(200, relu) + Dropout(0,5) + Dense(6, softmax)|Train from scratch|xx%|
|7|Size 64x64, batch size 128|Conv(4, relu) + Conv(8, relu) + MaxPooling(3) + Conv(16, relu) + MaxPooling(2) + Flatten + Dense(6, softmax)|Train from scratch|xx%|
|8|Size 64x64, batch size 128|Conv(4, relu) + Conv(8, relu) + MaxPooling(3) + Conv(16, relu) + MaxPooling(2) + Flatten + Dense(128, softmax) + Dropout(0,5) + Dense(6, softmax)|Train from scratch|xx%|
|9|Size 256, batch size 64|Conv(96, relu) + Conv(128, relu) + MaxPooling(3) + Conv(256, relu) + MaxPooling(3) + Conv(256, relu) + Conv(512, relu) + Conv(512, relu) + MaxPooling(2) + Flatten + Dense(512, relu) + Dense(6, softmax)|Train from scratch|xx%|
|10|Size 128, batch size 128|Conv(32, relu) + MaxPooling(2) + Conv(64, relu) + MaxPooling(2) + Conv(128, relu) + MaxPooling(2) + Conv(256, relu) + MaxPooling(2) + Flatten + Dense(4096, softmax) + Dropout(0,5) + Dense(256, softmax) + Dropout(0,5) + Dense(6, softmax)|Train from scratch|xx%|
|11|Size 128, batch size 128|Conv(32, relu) + MaxPooling(2) + Conv(64, relu) + MaxPooling(2) + Conv(128, relu) + MaxPooling(2) + Conv(256, relu) + MaxPooling(2) + Flatten + Dense(4096, softmax) + Dropout(0,5) + Dense(1024, softmax) + Dropout(0,5) + Dense(256, softmax) + Dropout(0,5) + Dense(6, softmax)|Train from scratch|xx%|
|12|Size 128, batch size 128|Conv(32, relu) + MaxPooling(2) + Conv(64, relu) + MaxPooling(2) + Conv(128, relu) + MaxPooling(2) + Conv(256, relu) + MaxPooling(2) + Flatten + Dense(2048, softmax) + Dropout(0,6) + Dense(1024, softmax) + Dropout(0,6) + Dense(256, softmax) + Dropout(0,5) + Dense(6, softmax)|Train from scratch|xx%|
|13|Size 128, batch size 128|Conv(32, relu) + MaxPooling(2) + Conv(64, relu) + MaxPooling(2) + Conv(128, relu) + MaxPooling(2) + Conv(256, relu) + MaxPooling(2) + Flatten + Dense(2048, softmax) + Dropout(0,6) + Dense(1024, softmax) + Dropout(0,6) + Dense(256, softmax) + Dropout(0,5) + Dense(6, softmax)|Train from scratch|xx%|


Conv(128, relu) + MaxPooling(3) + Conv(256, relu) + MaxPooling(3) + Conv(256, relu) + Conv(512, relu) + Conv(512, relu) + MaxPooling(2) + Flatten + Dense(512, relu) + Dense(6, softmax)|Train from scratch|xx%|

|Size 64x64, batch size 32|VGG16 + Flatten + Dense(32)|Bottleneck features|yy%|
|...|...|...|...|